In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("nyc_taxi").config("spark.local.dir", "/home/tb24/projects/Datalake-project").getOrCreate()

In [ ]:
df = spark.read.parquet("/home/tb24/projects/Datalake-project/data/yellow_tripdata_2023-03.parquet")
df.printSchema()

In [ ]:
df.createOrReplaceGlobalTempView("dfTable")

In [ ]:
from pyspark.sql.functions import avg
# Average fee of a taxi ride
df.selectExpr("avg(total_amount)").show(2)
# Average trip Distance
df.select(avg("trip_distance")).show(2)
# Average tips, tolls and fare and congestion surcharge
df.select(avg("tip_amount").alias("average_tip_amount"),
          avg("tolls_amount").alias('average_toll_amount'),
          avg("fare_amount").alias('average_fare'),
          avg("congestion_surcharge").alias('average_congestion_surcharge')
).show(2)

In [ ]:
from pyspark.sql.functions import min, max, count_distinct, col
# Biggest number of Passengers
df.select(max('passenger_count')).show(2)
# Biggest fare for a ride and its trip distance
df.groupby('trip_distance').agg(max('fare_amount')).show(2)